In [4]:
from keras.models import load_model

# Load the previously saved model
loaded_model = load_model('/workspaces/Cotton-Disease-Recognition-using-YOLO-Algorithm-/cotton_disease_mobilenet.h5')

2025-01-01 20:14:42.472062: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the previously saved model
loaded_model = load_model('/workspaces/Cotton-Disease-Recognition-using-YOLO-Algorithm-/cotton_disease_mobilenet.h5')

def predict_image(image_path, class_names, model):
    """
    Predicts the class of an image using the trained model.

    Args:
        image_path (str): Path to the image to predict.
        class_names (list): List of class labels.
        model (Model): The trained model used for prediction.

    Returns:
        str: Predicted class label.
        float: Confidence score of the prediction.
    """
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))  # Replace with your model's input size
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array)
    print(f"Raw predictions: {predictions}")  # Debugging output

    # Get the predicted class index and confidence
    predicted_index = np.argmax(predictions[0])
    confidence = predictions[0][predicted_index]

    # Check if predicted index is valid
    if predicted_index >= len(class_names):
        raise ValueError(
            f"Predicted index {predicted_index} exceeds the number of class labels in class_names {len(class_names)}."
        )

    # Map predicted index to class label
    predicted_class = class_names[predicted_index]

    return predicted_class, confidence

# Define the class names
class_names = ['fresh cotton leaf', 'fresh cotton plant', 'diseased cotton leaf', 'diseased cotton plant']

# Example: Predict for a sample image
image_path = "/workspaces/Cotton-Disease-Recognition-using-YOLO-Algorithm-/cotton2.jpeg"
predicted_class, confidence = predict_image(image_path, class_names, loaded_model)

print(f"Predicted Class: {predicted_class}")
print(f"Confidence Score: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 675ms/step
Raw predictions: [[0.03159545 0.00178412 0.9547751  0.01184542]]
Predicted Class: diseased cotton leaf
Confidence Score: 0.95


In [ ]:
import cv2
import os

def draw_bounding_boxes(image_path, label_path, class_names=None):
    """
    Draws bounding boxes on the image using YOLO label format.

    :param image_path: Path to the image file
    :param label_path: Path to the label file (YOLO format)
    :param class_names: List of class names (optional)
    """
    # Load image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Read bounding boxes from label file
    if not os.path.exists(label_path):
        print(f"Label file {label_path} not found!")
        return

    with open(label_path, "r") as file:
        lines = file.readlines()

    for line in lines:
        values = line.strip().split()
        class_id = int(values[0])
        x_center, y_center, w, h = map(float, values[1:])

        # Convert YOLO format (normalized) to absolute pixel values
        x1 = int((x_center - w / 2) * width)
        y1 = int((y_center - h / 2) * height)
        x2 = int((x_center + w / 2) * width)
        y2 = int((y_center + h / 2) * height)

        # Draw rectangle and label
        color = (0, 255, 0)  # Green bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        
        label = class_names[class_id] if class_names else f"Class {class_id}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show image with bounding boxes
    cv2.imshow("Image with Bounding Boxes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = "/Users/shubhampund9767/Documents/Cotton-Disease-Recognition-using-YOLO-Algorithm--1/Cotton-Disease-Specific/trainning/images/Green Cotton Boll/1.jpg"
label_path = "/Users/shubhampund9767/Documents/Cotton-Disease-Recognition-using-YOLO-Algorithm--1/Cotton-Disease-Specific/trainning/labels/Green Cotton boll/1.txt"
class_names = ["Aphid","Cotton Healthy", "Cotton Diseased","Green Cotton boll"]  # Example class names

draw_bounding_boxes(image_path, label_path, class_names)

2025-02-07 15:43:42.870 python[33244:4077105] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-07 15:43:42.870 python[33244:4077105] +[IMKInputSession subclass]: chose IMKInputSession_Modern
